In [1]:
!pip install sentence-transformers
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f0ceb3dda3681590e32afd6317cbf46d94e07f64db0e9038ce85a47b060b85c0
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
import re
import nltk
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
def calculate_similarity(sentences, unique_sentences):
    similarities = []
    for sentence in sentences:
        if unique_sentences:
            embeddings = model.encode([sentence] + unique_sentences)
            similarity_scores = cosine_similarity([embeddings[0]], embeddings[1:])[0]
            max_similarity = max(similarity_scores)
            if max_similarity < 0.8:
                unique_sentences.append(sentence)
        else:
            unique_sentences.append(sentence)

    return unique_sentences

In [5]:
def is_potential_name(word):
    return word[0].isupper() and word[1:].islower()

In [6]:
def case_formatter(review):
    review = re.sub(r'\.{2,}', '.', review)
    sentences = nltk.sent_tokenize(review)
    formatted_sentences = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        formatted_words = []
        for i, word in enumerate(words):
            if i == 0 or not is_potential_name(word):
                word = word.lower()
            formatted_words.append(word)
        formatted_sentence = ' '.join(formatted_words)
        formatted_sentence = formatted_sentence.capitalize()
        formatted_sentences.append(formatted_sentence)
    formatted_review = '. '.join(formatted_sentences)
    formatted_review += '.'
    return formatted_review

In [7]:


def clean_reviews(review):
    review = re.sub(r'\.{2,}', '.', review)
    sentences = review.split('.')
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    unique_sentences = list(set(sentences))
    cleaned_review = '. '.join(unique_sentences)
    return cleaned_review + "."

In [8]:
def summary_finalizer(dataframe, batch_size):
    dataframe = dataframe.sort_values(by='Sequence')  # Sort dataframe by sequence number

    batch_strings_list = []
    unique_strings_list = []
    final_cleaned_strings_list = []

    for i in range(0, len(dataframe), batch_size):
        batch = dataframe[i:i+batch_size]
        batch_strings = list(batch['Summeries column'])  # Get the strings from the 'Summeries column' in sequence number order
        batch_strings_list.extend(batch_strings)

        unique_strings = calculate_similarity(batch_strings, unique_strings_list)  # Pass the batch strings through the similarity calculation
        unique_strings_list.extend(unique_strings)

    for string in unique_strings_list:
        formatted_string = case_formatter(string)
        cleaned_string = clean_reviews(formatted_string)
        final_cleaned_strings_list.append(cleaned_string)

    cleaned_dataframe = pd.DataFrame(columns=['Sequence',  'Summeries column', 'Cleaned summary'])

    for i, sequence in enumerate(dataframe['Sequence']):
        # review = dataframe.loc[dataframe['Sequence'] == sequence, 'review'].iloc[0]
        summer_or_review = dataframe.loc[dataframe['Sequence'] == sequence, 'Summeries column'].iloc[0]
        cleaned_summary = final_cleaned_strings_list[i]

        cleaned_dataframe = cleaned_dataframe.append({'Sequence': sequence,
                                                      # 'review': review,
                                                      'Summeries column': summer_or_review,
                                                      'Cleaned summary': cleaned_summary},
                                                     ignore_index=True)
    return cleaned_dataframe


In [9]:
import pandas as pd

summaries = [
    "a tsxi. a tsxi......... The wine...... The staff could smile... The staff could smile. The pool.. The staff could smile... Room view was of generator..",
    ". Very small. Very noisy air conditioning. Pool. The room. Location. The room. Wi Fi connection poor. Pool. The room. Location. The room. Location. The room. Location. The room. Location. The room. Location. The room. Location. The room. Location. The room. Location. The room. Location. The room. Location. The room. Location. The room. Location. The room.",
    "....... No tea coffee coffee or water in room. No tea coffee or water in room. No tea coffee or water in room. No tea coffee or water in room. No tea coffee or water in room. No tea coffee or water in room. No tea coffee or water in room. No tea coffee or water in room. No coffee. No water in room. No tea or coffee in room.. No tea coffee or water in room. No tea coffee.",
    "Poor service from staff Poor service from staff Emails not returned. No water hous. Poor service from staff Emails not returned. Poor service from staff Emails not returned. No water hous. Poor service from staff Emails not returned. Poor service from staff Emails not returned. Poor service from staff Emails not returned. Not much space in the rooms but expected that in central london.",
    "The staff were awful. The pool wifi was a bit too expensive The pool wifi was a bit too expensive. The pool wifi was a bit too expensive. Breakfast was not good.......... Bathrooms a bit small... Small bathroom. Small room. Small bathroom. Small bathroom. Small bathroom... Appalling customer service... .",
    "The staff were TERRIBLE UNHELPFUL SLOW AND WITH A COULDN t CARE LESS ATITUDE PARKING OF THE LACK OF IT WAS HORRENDOUS. Rooms were small. No wi fi in room when states on ad that it was standard size of room and ridiculous prices in bar.",
    "The staff were TERRIBLE UNHELPFUL SLOW AND WITH A COULDN t CARE LESS ATITUDE PARKING OF THE LACK OF IT WAS HORRENDOUS. Rooms were small. No wi fi in room when states on ad that it was standard size of room and ridiculous prices in bar.",
    ".. Bathroom to small shower was a Trickle of water. Bathroom to small shower was a Trickle of water. Bathroom very small....... Bathroom is small and dated........... Bathroom was tiny. Not applicable.. Bathroom was tiny.. Location..... Bathroom is small."
]

df = pd.DataFrame({'Summeries column': summaries})
df['Sequence'] =  [3,2,7,4,5,6,1,8]


In [10]:
batch_size = 9  # Adjust the batch size as per your requirement
cleaned_df = summary_finalizer(df, batch_size)
display(cleaned_df)

<ipython-input-8-8909528ee679>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_dataframe = cleaned_dataframe.append({'Sequence': sequence,
<ipython-input-8-8909528ee679>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_dataframe = cleaned_dataframe.append({'Sequence': sequence,
<ipython-input-8-8909528ee679>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_dataframe = cleaned_dataframe.append({'Sequence': sequence,
<ipython-input-8-8909528ee679>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_dataframe = cleaned_dataframe.append({'Sequence': sequence,
<ipython-input-8-8909528ee679>:28: FutureWarning

,Sequence,Summeries column,Cleaned summary
0,1,The staff were TERRIBLE UNHELPFUL SLOW AND WIT...,The staff were terrible unhelpful slow and wit...
1,2,. Very small. Very noisy air conditioning. Poo...,Very small. Location. Pool. The room. Wi fi co...
2,3,a tsxi. a tsxi......... The wine...... The sta...,Room view was of generator. The wine. The pool...
3,4,Poor service from staff Poor service from staf...,Poor service from staff poor service from staf...
4,5,The staff were awful. The pool wifi was a bit ...,Appalling customer service. The pool wifi was ...
5,6,The staff were TERRIBLE UNHELPFUL SLOW AND WIT...,No tea coffee or water in room. No water in ro...
6,7,....... No tea coffee coffee or water in room....,Location. Bathroom very small. Bathroom is sma...
7,8,.. Bathroom to small shower was a Trickle of w...,The staff were terrible unhelpful slow and wit...


In [11]:
display(cleaned_df)

,Sequence,Summeries column,Cleaned summary
0,1,The staff were TERRIBLE UNHELPFUL SLOW AND WIT...,The staff were terrible unhelpful slow and wit...
1,2,. Very small. Very noisy air conditioning. Poo...,Very small. Location. Pool. The room. Wi fi co...
2,3,a tsxi. a tsxi......... The wine...... The sta...,Room view was of generator. The wine. The pool...
3,4,Poor service from staff Poor service from staf...,Poor service from staff poor service from staf...
4,5,The staff were awful. The pool wifi was a bit ...,Appalling customer service. The pool wifi was ...
5,6,The staff were TERRIBLE UNHELPFUL SLOW AND WIT...,No tea coffee or water in room. No water in ro...
6,7,....... No tea coffee coffee or water in room....,Location. Bathroom very small. Bathroom is sma...
7,8,.. Bathroom to small shower was a Trickle of w...,The staff were terrible unhelpful slow and wit...
